# Bonus: two-item stochastic pricing environment

### 1. Import

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Suppress warnings

### 2. Two-item pricing environment

In [ ]:
class TwoItemPricing:
    def __init__(self, T, noise_std=5, cost1=1, cost2=1, correlation=0.5):
        """
        Initialize the TwoItemPricing class.

        Parameters:
        - T: Time horizon
        - noise_std: Standard deviation of the Gaussian noise
        - cost1: Cost of producing the first item
        - cost2: Cost of producing the second item
        - correlation: Correlation between the demands of the two items
        """
        self.T = T
        self.noise_std = noise_std
        self.cost1 = cost1
        self.cost2 = cost2
        self.correlation = correlation
        self.prices = np.linspace(0, 1, 50)  # Discretized prices for plotting

        # True demand parameters for the two items
        self.a1 = 100
        self.b1 = 50
        self.a2 = 80
        self.b2 = 40

        # Gaussian Process models for the two items
        kernel = C(1.0, (1e-3, 1e3)) * RBF(1, (1e-2, 1e2))
        self.gp1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=noise_std**2)
        self.gp2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=noise_std**2)

        self.X = []
        self.y1 = []
        self.y2 = []
        self.selected_prices = []
        self.rewards = []

    def demand_curve(self, p1, p2):
        """
        Calculate the demand for given prices of two items.
        - p1: Price of the first item
        - p2: Price of the second item
        Returns:
        - d1: Demand for the first item with added correlated noise
        - d2: Demand for the second item with added correlated noise
        """
        cov_matrix = [[self.noise_std**2, self.correlation * self.noise_std**2],
                      [self.correlation * self.noise_std**2, self.noise_std**2]]
        noise = np.random.multivariate_normal([0, 0], cov_matrix)

        d1 = self.a1 - self.b1 * p1 + noise[0]
        d2 = self.a2 - self.b2 * p2 + noise[1]
        return d1, d2

    def simulate(self):
        """
        Simulate the environment to generate rewards for each price pair at each time step.
        """
        for t in range(self.T):
            p1, p2 = np.random.uniform(0, 1, 2)
            d1, d2 = self.demand_curve(p1, p2)
            profit1 = (p1 - self.cost1) * d1
            profit2 = (p2 - self.cost2) * d2

            self.X.append([p1, p2])
            self.y1.append(profit1)
            self.y2.append(profit2)

            self.gp1.fit(np.array(self.X), np.array(self.y1))
            self.gp2.fit(np.array(self.X), np.array(self.y2))

    def regret_minimizer(self):
        """
        Implement the regret minimizer using GP-UCB.

        Returns:
        - selected_prices: List of price pairs selected at each time step
        - total_reward: Total reward accumulated
        """
        total_reward = 0
        beta = 2.0  # Exploration-exploitation parameter

        for t in range(self.T):
            # Predict the mean and standard deviation of the reward functions
            X_test = np.array([[p1, p2] for p1 in self.prices for p2 in self.prices])
            mu1, sigma1 = self.gp1.predict(X_test, return_std=True)
            mu2, sigma2 = self.gp2.predict(X_test, return_std=True)

            # Compute the UCB values
            ucb_values = mu1 + beta * sigma1 + mu2 + beta * sigma2
            best_idx = np.argmax(ucb_values)
            best_price = X_test[best_idx]
            self.selected_prices.append(best_price)

            # Simulate the reward for the selected prices
            d1, d2 = self.demand_curve(best_price[0], best_price[1])
            profit1 = (best_price[0] - self.cost1) * d1
            profit2 = (best_price[1] - self.cost2) * d2
            total_reward += profit1 + profit2
            self.rewards.append(profit1 + profit2)

            # Update the GP models with the new data
            self.X.append(best_price)
            self.y1.append(profit1)
            self.y2.append(profit2)
            self.gp1.fit(np.array(self.X), np.array(self.y1))
            self.gp2.fit(np.array(self.X), np.array(self.y2))

        return self.selected_prices, total_reward

    def plot_demand_curves_with_noise(self):
        """
        Plot the demand curves for the two items with noise.
        """
        p1 = np.linspace(0, 1, 100)
        d1 = self.a1 - self.b1 * p1
        d2 = self.a2 - self.b2 * p1
        d1_noisy = d1 + np.random.normal(0, self.noise_std, size=p1.shape)
        d2_noisy = d2 + np.random.normal(0, self.noise_std, size=p1.shape)

        plt.figure(figsize=(12, 6))
        plt.plot(p1, d1, label='Demand for Item 1 (true)')
        plt.plot(p1, d1_noisy, label='Demand for Item 1 (noisy)', linestyle='dashed')
        plt.plot(p1, d2, label='Demand for Item 2 (true)')
        plt.plot(p1, d2_noisy, label='Demand for Item 2 (noisy)', linestyle='dashed')
        plt.xlabel('Price')
        plt.ylabel('Demand')
        plt.title('Demand Curves for Two Items with Noise')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_rewards_and_prices(self):
        """
        Plot the rewards and prices over time.
        """
        times = range(self.T)
        prices1 = [p[0] for p in self.selected_prices]
        prices2 = [p[1] for p in self.selected_prices]

        plt.figure(figsize=(12, 6))
        plt.subplot(2, 1, 1)
        plt.plot(times, prices1, label='Price of Item 1')
        plt.plot(times, prices2, label='Price of Item 2')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.title('Prices Over Time')
        plt.legend()
        plt.grid(True)

        plt.subplot(2, 1, 2)
        plt.plot(times, self.rewards, label='Rewards')
        plt.xlabel('Time')
        plt.ylabel('Reward')
        plt.title('Rewards Over Time')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

# Parameters
T = 100  # Time horizon
noise_std = 2  # Standard deviation of the noise
cost1 = 1  # Cost of producing the first item
cost2 = 1  # Cost of producing the second item
correlation = 0.5  # Correlation between the demands of the two items

# Initialize and simulate the environment
pricing = TwoItemPricing(T, noise_std, cost1, cost2, correlation)
pricing.simulate()
selected_prices, total_reward = pricing.regret_minimizer()

print(f"Total Reward: {total_reward}")
print(f"Selected Prices: {selected_prices}")

# Plot demand curves with noise
pricing.plot_demand_curves_with_noise()

# Plot rewards and prices over time
pricing.plot_rewards_and_prices()


### Regret Minimizer
Build a regret minimizer for the continuous action set [0, 1] using two-dimensional
Gaussian processes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

class TwoItemPricing:
    def __init__(self, T, noise_std=5, cost1=1, cost2=1, correlation=0.5):
        """
        Initialize the TwoItemPricing class.

        Parameters:
        - T: Time horizon
        - noise_std: Standard deviation of the Gaussian noise
        - cost1: Cost of producing the first item
        - cost2: Cost of producing the second item
        - correlation: Correlation between the prices of the two items
        """
        self.T = T
        self.noise_std = noise_std
        self.cost1 = cost1
        self.cost2 = cost2
        self.correlation = correlation
        self.prices = np.linspace(0, 1, 50)  # Discretized prices for plotting

        # True demand parameters for the two items
        self.a1 = 100
        self.b1 = 50
        self.a2 = 80
        self.b2 = 40

        # Gaussian Process models for the two items
        kernel = C(1.0, (1e-3, 1e3)) * RBF(1, (1e-2, 1e2))
        self.gp1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=noise_std**2)
        self.gp2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=noise_std**2)

        self.X = []
        self.y1 = []
        self.y2 = []
        self.selected_prices = []
        self.rewards = []

        # Covariance matrix for correlated prices
        self.cov_matrix = [[1, self.correlation],
                           [self.correlation, 1]]

    def correlated_prices(self):
        """
        Generate correlated prices.

        Returns:
        - p1: Price of the first item
        - p2: Price of the second item
        """
        prices = np.random.multivariate_normal([0.5, 0.5], self.cov_matrix)
        return np.clip(prices, 0, 1)

    def demand_curve(self, p1, p2):
        """
        Calculate the demand for given correlated prices of two items.

        Parameters:
        - p1: Price of the first item
        - p2: Price of the second item

        Returns:
        - d1: Demand for the first item with added noise
        - d2: Demand for the second item with added noise
        """
        d1 = self.a1 - self.b1 * p1 + np.random.normal(0, self.noise_std)
        d2 = self.a2 - self.b2 * p2 + np.random.normal(0, self.noise_std)
        return d1, d2

    def simulate(self):
        """
        Simulate the environment to generate rewards for each price pair at each time step.
        """
        for t in range(self.T):
            p1, p2 = self.correlated_prices()
            d1, d2 = self.demand_curve(p1, p2)
            profit1 = (p1 - self.cost1) * d1
            profit2 = (p2 - self.cost2) * d2

            # Debugging print statements
            print(f"Time step {t}: p1={p1:.2f}, p2={p2:.2f}, d1={d1:.2f}, d2={d2:.2f}, profit1={profit1:.2f}, profit2={profit2:.2f}")

            self.X.append([p1, p2])
            self.y1.append(profit1)
            self.y2.append(profit2)

            self.gp1.fit(np.array(self.X), np.array(self.y1))
            self.gp2.fit(np.array(self.X), np.array(self.y2))

    def regret_minimizer(self):
        """
        Implement the regret minimizer using GP-UCB.

        Returns:
        - selected_prices: List of price pairs selected at each time step
        - total_reward: Total reward accumulated
        """
        total_reward = 0
        beta = 2.0  # Exploration-exploitation parameter

        for t in range(self.T):
            # Predict the mean and standard deviation of the reward functions
            X_test = np.array([[p1, p2] for p1 in self.prices for p2 in self.prices])
            mu1, sigma1 = self.gp1.predict(X_test, return_std=True)
            mu2, sigma2 = self.gp2.predict(X_test, return_std=True)

            # Compute the UCB values
            ucb_values = mu1 + beta * sigma1 + mu2 + beta * sigma2
            best_idx = np.argmax(ucb_values)
            best_price = X_test[best_idx]
            self.selected_prices.append(best_price)

            # Simulate the reward for the selected prices
            d1, d2 = self.demand_curve(best_price[0], best_price[1])
            profit1 = (best_price[0] - self.cost1) * d1
            profit2 = (best_price[1] - self.cost2) * d2
            total_reward += profit1 + profit2
            self.rewards.append(profit1 + profit2)

            # Debugging print statements
            print(f"Time step {t}: selected p1={best_price[0]:.2f}, selected p2={best_price[1]:.2f}, d1={d1:.2f}, d2={d2:.2f}, profit1={profit1:.2f}, profit2={profit2:.2f}")

            # Update the GP models with the new data
            self.X.append(best_price)
            self.y1.append(profit1)
            self.y2.append(profit2)
            self.gp1.fit(np.array(self.X), np.array(self.y1))
            self.gp2.fit(np.array(self.X), np.array(self.y2))

        return self.selected_prices, total_reward

    def plot_demand_curves_with_noise(self):
        """
        Plot the demand curves for the two items with noise.
        """
        p1 = np.linspace(0, 1, 100)
        d1 = self.a1 - self.b1 * p1
        d2 = self.a2 - self.b2 * p1
        d1_noisy = d1 + np.random.normal(0, self.noise_std, size=p1.shape)
        d2_noisy = d2 + np.random.normal(0, self.noise_std, size=p1.shape)

        plt.figure(figsize=(12, 6))
        plt.plot(p1, d1, label='Demand for Item 1 (true)')
        plt.plot(p1, d1_noisy, label='Demand for Item 1 (noisy)', linestyle='dashed')
        plt.plot(p1, d2, label='Demand for Item 2 (true)')
        plt.plot(p1, d2_noisy, label='Demand for Item 2 (noisy)', linestyle='dashed')
        plt.xlabel('Price')
        plt.ylabel('Demand')
        plt.title('Demand Curves for Two Items with Noise')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_rewards_and_prices(self):
        """
        Plot the rewards and prices over time.
        """
        times = range(self.T)
        prices1 = [p[0] for p in self.selected_prices]
        prices2 = [p[1] for p in self.selected_prices]

        plt.figure(figsize=(12, 6))
        plt.subplot(2, 1, 1)
        plt.plot(times, prices1, label='Price of Item 1')
        plt.plot(times, prices2, label='Price of Item 2')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.title('Prices Over Time')
        plt.legend()
        plt.grid(True)

        plt.subplot(2, 1, 2)
        plt.plot(times, self.rewards, label='Rewards')
        plt.xlabel('Time')
        plt.ylabel('Reward')
        plt.title('Rewards Over Time')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

    def print_covariance_matrix(self):
        """
        Print the covariance matrix used to generate correlated prices.
        """
        print("Covariance Matrix for Prices:")
        print(np.array(self.cov_matrix))

# Parameters
T = 100  # Time horizon
noise_std = 2  # Standard deviation of the noise
cost1 = 1  # Cost of producing the first item
cost2 = 1  # Cost of producing
